# JS customization

Table Of Contents :

+ [Custom Buttons](#Building-custom-buttons)
+ [Custom JS](#Completely-custom-js)
+ [Linking Widgets](#Linking-widgets-together)
    - [Watching updates from outside the grid](#Updating-information-in-the-grid-via-outside-widgets)
    - [Watching the data of the grid for interactive plotting](#Using-the-interactivity-of-the-grid-outputs-for-interactive-plotings)

In [1]:
import os
import json
import numpy as np
import pandas as pd
import urllib.request as ur
import ipywidgets as widgets
from ipyaggrid import Grid
from IPython.display import display

In [2]:
url = './data/olympicWinners.json'
with open(url, encoding='utf-8') as f:
    data = json.loads(f.read())

In [3]:
# import sys
# import json
# from pygments import highlight, lexers, formatters

# formatted_json = json.dumps(data, indent=4)
# colorful_json = highlight(formatted_json, lexers.JsonLexer(), formatters.TerminalFormatter())
# print(formatted_json)

In [4]:
pd.DataFrame(data).head(5)

,age,athlete,bronze,country,date,gold,silver,sport,total,year
0,23.0,Michael Phelps,0,United States,24/08/2008,8,0,Swimming,8,2008
1,19.0,Michael Phelps,2,United States,29/08/2004,6,0,Swimming,8,2004
2,27.0,Michael Phelps,0,United States,12/08/2012,4,2,Swimming,6,2012
3,25.0,Natalie Coughlin,3,United States,24/08/2008,1,2,Swimming,6,2008
4,24.0,Aleksey Nemov,3,Russia,01/10/2000,2,1,Gymnastics,6,2000


## Building custom buttons

Buttons are an easy way to interact with the grid. They can perform many actions on the gridOptions, and many examples of them are already given in [ag-Grid documentation](https://www.ag-grid.com/documentation-main/documentation.php), so that you can test anyfeature you want using their Plunckers (in browsers test environments).

This custom button is built using two parameters : the name of the button and the action of the button. The first one will be used to be displayed on the button, and the second one is the body of the function called on click.
The code **can use the `gridOptions`** as if they were defined before, and thus access to the `api` and the `columnApi` of ag-Grid.

With this custom button, two cells are highlighted if their difference is higher than 50.

In [5]:
columnDefs = [
    {'headerName': "Country", 'field': "country", 'width': 120, 'rowGroup': 'true'},
    {'headerName': "Year", 'field': "year", 'width': 90, 'pivot': 'true', 'enablePivot':True},
    {'headerName': "Sport", 'field': "sport", 'width': 110, 'rowGroup': 'true'},
    {'headerName': "Athlete", 'field': "athlete"},
    {'headerName': "Gold", 'field': "gold", 'width': 100, 'aggFunc': 'sum'},
];

gridOptions = {
    'pivotMode': 'true',
    'enableColResize': 'true',
    'columnDefs': columnDefs,
    'enableFilter':'true',
    'enableSorting':'true',
    'animateRows':'true',
};

buttons=[{'name':'Highlight', 'action':"""
        var count = gridOptions.api.getDisplayedRowCount();
        for (var i = 0; i<count; i++) {
          var rowNode = gridOptions.api.getDisplayedRowAtIndex(i);
          if(rowNode.aggData != null && Object.keys(rowNode.aggData).length > 0){
        var keys = Object.keys(rowNode.aggData);
        var gold = [];
        for (var k = 0; k<keys.length; k++){
          var j = 2*k + 1;
          var prop = "pivot_" + j;
          if(rowNode.aggData[prop] == null){
            rowNode.aggData[prop] = 0;
          }
          gold[k] = rowNode.aggData[prop];
        }
        for(var j=0;j<gold.length - 1;j++){
          if(Math.abs(gold[j] - gold[j+1]) >= 50){
            var column1 = "pivot_" + (2*j+1);
            var column2 = "pivot_" + (2*(j+1)+1);
            gridOptions.api.flashCells({rowNodes: [rowNode], columns: [column1, column2] });
          }
        }}}"""}]


grid1 = Grid(quick_filter=True,
             theme='ag-theme-balham',
             compress_data=True,
             menu={'buttons':buttons},
             grid_options=gridOptions,
             grid_data=data,
             columns_fit="auto")
grid1

Grid(columns_fit='auto', compress_data=True, export_mode='disabled', height='350px', menu={'buttons': [{'name'…

## Completely custom js - Pay attention: <span style="color:red">A lot of tips here</span>

The example below shows how to:
+ Insert a column computed from the input data
+ Set [cell style](https://www.ag-grid.com/javascript-grid-cell-styling/):
    + **Age** 30 and above is ahow in red
+ Set [cell renderers](https://www.ag-grid.com/javascript-grid-cell-rendering/):
    + **Sport** is displayed with a button: Open console to see the data collected upon click
    + **Score** is displayed with bar in the background 
+ Use the [**ag-Grid** context object](https://www.ag-grid.com/javascript-grid-context/) to carry precalculated and grid (as opposed to row) level info
+ Use the various JS injection points:
    + `js_helpers_custom`: Define a new helper function
    + `js_pre_grid`: Enrich ColumnDefs before grid instantiation
    + `js_post_grid`: Determine values (`max_score`) and functions (`scale`) from all data
    
    
+ A **score** is calculated from several columns:
    + $\sqrt{\displaystyle(\text{3 gold}^2+\text{2 silver}^2+\text{1 bronze}^2)}$. 
    + **Note**: It does not represent any real world value, just a pretext for the background bars. 


In [6]:
%%javascript

require.config({
    paths: {
        'plotly'  : 'https://cdn.plot.ly/plotly-latest.min',
        'echarts' : 'https://cdnjs.cloudflare.com/ajax/libs/echarts/4.2.1/echarts.min',
        'moment'  : 'https://cdnjs.cloudflare.com/ajax/libs/moment.js/2.24.0/moment.min'
    }
});

require(['plotly', 'echarts', 'moment'], function(plotly, echarts, moment) {
    window.echarts = echarts;
});

<IPython.core.display.Javascript object>

In [7]:
age_cell_style = """
function(params) { if (params.value >= 30) { return {color: 'red'}; } }
"""

sport_cell_renderer = """
function (params) {
    try {
        let v = params.value;

        function clicked() {
            //window.params = params;
            let scale = params.api.context.data_info['score'].scale;

            console.log('row data:');
            console.log(params.data);
            console.log('row index:');
            console.log(params.rowIndex);
            console.log('score:');
            console.log(params.data.score);
            console.log('max_score (from context):');
            console.log(params.api.context.data_info['score'].max_v);
        }

        let b = document.createElement('select');
        b.innerHTML = '<option>AAA</option><option>BBB</option><option>CCC</option>';
        b.className = 'btn btn-success btn-xs';
        b.style = "margin: 1px 10px 1px 2px;";
        //b.title = "Open console after click";
        //b.addEventListener("click", function (){clicked()}, false);

        let d = document.createElement('div');
        d.style = 'display: flex';
        let d2 = document.createElement('div');
        d2.innerHTML = v;
        d.appendChild(b);
        d.appendChild(d2);

        return d;
    } catch(e) {
        return params.value;
    }
}
"""

columnDefs = [
    {
        'headerName': "Athlete", 'field': "athlete", 'width': 150, 'enableRowGroup': True, 
        'enablePivot': True, 'rowDrag': True, 'checkboxSelection': True
    },
    {
        'headerName': "Age", 'field': "age", 'width': 90, 'enableRowGroup': True, 
        'enablePivot': True, 'cellRenderer': 'helpers.ageRendering', 'pinnedRowCellRenderer': 'customPinnedRowRenderer'
    },
#     {'headerName': "Country", 'field': "country", 'width': 120},
    {
        'headerName': "Year", 'field': "year", 'width': 90, 'enableRowGroup': True, 'enablePivot': True
    },
#     {'headerName': "Date", 'field': "date", 'width': 145},
    {
        'headerName': "Sport", 'field': "sport", 'width': 180, 'enableRowGroup': True, 'enablePivot': True, 'cellRenderer': sport_cell_renderer
    },
    {
        'headerName': "Gold", 'field': "gold", 'width': 70, 'cellRenderer': 'helpers.myCellRenderer'
    },
    {
        'headerName': "Silver", 'field': "silver", 'width': 75, 'enableRowGroup': True, 
        'enablePivot': True, 'enableValue' : True, 'allowedAggFuncs': ['sum','min','max'], 'pinnedRowCellRenderer': 'customPinnedRowRenderer'
    },
    {
        'headerName': "Bronze", 'field': "bronze", 'width': 85, 'enableRowGroup': True, 
        'enablePivot': True, 'enableValue' : True, 'allowedAggFuncs': ['sum','min','max'], 'pinnedRowCellRenderer': 'customPinnedRowRenderer'
    }
]

# JS injection
# Ref: https://gitlab.com/DGothrek/ipyaggrid/blob/master/js/src/widget_builder.js#L61
js_helpers_custom="""
helpersCustom.ageRendering = function(params) {    
    let dom = document.createElement('input');
    dom.type     = 'range';
    dom.min      = 10;
    dom.max      = 50;
    dom.step     = 1;
    dom.value    = params.value;
    dom.onchange = function() {
        params.setValue(this.value);
    };
    return dom;
};

helpersCustom.goldCellRenderer = function(params) {
    
    var wrap_dom = document.createElement('div');
    wrap_dom.style = 'width:100%;height:100%;';
    
    setTimeout(function() {
    
        require(['plotly', 'echarts', 'moment'], function(Plotly, echarts, moment) {

            var myChart = echarts.init(wrap_dom);

            var option = {
                title: {
                    text: 'ECharts 入门示例'
                },
                tooltip: {},
                legend: {
                    data:['销量']
                },
                xAxis: {
                    data: ["衬衫","羊毛衫","雪纺衫","裤子","高跟鞋","袜子"]
                },
                yAxis: {},
                series: [{
                    name: '销量',
                    type: 'bar',
                    data: [5, 20, 36, 10, 10, 20]
                }]
            };

            myChart.setOption(option);

        });
        
    }, 0);
    
    return wrap_dom;
}

class MyCellRenderer {
    
    constructor() {
    
    }
    
    init(params) {        
        this.eGui = document.createElement('div');
        this.eGui.style = 'width:100%;height:100%;';
        
        //setTimeout(() => {
            
            this.myChart = echarts.init(this.eGui);

            let option = {
                title: {
                    text: 'ECharts 入门示例'
                },
                tooltip: {},
                legend: {
                    data:['销量']
                },
                xAxis: {
                    data: ["衬衫","羊毛衫","雪纺衫","裤子","高跟鞋","袜子"]
                },
                yAxis: {},
                series: [{
                    name: '销量',
                    type: 'bar',
                    data: [5, 20, 36, 10, 10, 20]
                }]
            };

            this.myChart.setOption(option);

        //}, 0);
    }
    
    getGui() {
        return this.eGui;
    }
    
    refresh(params) {
        this.eValue.innerHTML = params.valueFormatted ? params.valueFormatted : params.value;
        return true;
    }
    
    destroy() {
        if (this.myChart && !this.myChart.isDisposed()) {
            this.myChart.clear();
            this.myChart.dispose();
        }
    }
}

helpersCustom.myCellRenderer = MyCellRenderer;
"""

# JS injection
# Ref: https://gitlab.com/DGothrek/ipyaggrid/blob/master/js/src/widget_builder.js#L103
js_pre_grid = ["""

function scoreCellRenderer(params) {
    try {
        let v = params.value;
        let scale = params.api.context.data_info['score'].scale;
        let f = scale(v);
        let css = `
            background: linear-gradient(to right, #e0a800 ${f}%, transparent ${f}%); 
            flex-grow: 2
        `;
        html = `<div style="${css}">${v.toFixed(2)}</div>`
    } catch (e) {
        html = ''
    }
    return html;
}

function CustomPinnedRowRenderer () {}
CustomPinnedRowRenderer.prototype.init = function(params) {
        
    var eGui = document.createElement('div');
    eGui.id = 'header_sum_' + params.column.getId();
    this.eGui = eGui;
    
    setTimeout(function() {
        /*
        $(eGui).sparkline(params.value, {
            type: 'box',
            width: 60,
            height: 20,
            tooltipClassname: 'myjqstooltip'
        });
        */
    }, 0);
    
};
CustomPinnedRowRenderer.prototype.getGui = function() {
    return this.eGui;
};


gridData.forEach(dat => {
    dat.score = Math.sqrt(Math.pow(3*dat.gold,2) + Math.pow(2*dat.silver,2) + Math.pow(1*dat.bronze,2))
})

function gen_array(col_name) {
    let arr = [];
    gridData.forEach(dat => {arr.push(dat[col_name])});
    return arr;
}

//gridOptions.columnDefs.push({'headerName': "Gold", 'field': "gold", 'width': 70, cellRenderer: helpersCustom.goldCellRenderer, pinnedRowCellRenderer: 'customPinnedRowRenderer'});
gridOptions.columnDefs.push({'headerName':'Score', field:'score', cellRenderer: scoreCellRenderer, pinnedRowCellRenderer: 'customPinnedRowRenderer'});
gridOptions.pinnedTopRowData = [{
    age: gen_array('age'),
    year: gen_array('year'),
    sport: gen_array('sport'),
    gold: gen_array('gold'),
    silver: gen_array('silver'),
    bronze: gen_array('bronze'),
    score: gen_array('score')
}];

gridOptions['components'] = {
    customPinnedRowRenderer: CustomPinnedRowRenderer
}
/*
gridOptions.getRowHeight = function(params) {
    if (params.node.rowPinned) {
        return 26;
    } else {
        return 20;
    }
}
*/
gridOptions['getMainMenuItems'] = function(params) {
    switch (params.column.getId()) {
        case 'athlete':
            var athleteMenuItems = params.defaultItems.slice(0);
            athleteMenuItems.push({
                name: '[Custom] ag-Grid Is Great',
                icon: '<i class="fa fa-check"></i>',
                action: function() {
                    $('#header_sum_age').html('321');
                }
            });
            athleteMenuItems.push({
                name: '[Custom] Casio Watch',
                icon: '<i class="fa fa-id-card"></i>',
                action: function() {
                    console.log('People who wear casio watches are cool');
                }
            });
            athleteMenuItems.push({
                name: '[Custom] Custom Sub Menu',
                subMenu: [
                    {name: 'Black', action: function() {console.log('Black was pressed');} },
                    {name: 'White', action: function() {console.log('White was pressed');} },
                    {name: 'Grey', action: function() {console.log('Grey was pressed');} }
                ]
            });
            return athleteMenuItems;
        default:
            return params.defaultItems;
    }
}
"""]

# JS injection
# Ref: https://gitlab.com/DGothrek/ipyaggrid/blob/master/js/src/widget_builder.js#L121
js_post_grid = ["""
window.go = gridOptions;

function getDataInfo(col_name) {
    let arr_score = [], min_v = 0, max_v = 0, scale = 0;

    gridOptions.api.forEachNode(node => {arr_score.push(node.data[col_name])});
    [min_v, max_v] = d3.extent(arr_score);
    scale = d3.scaleLinear().domain([min_v, max_v]).range([0, 100]);
    
    return {scale: scale, min_v: min_v, max_v: max_v};
}

gridOptions.api.context.data_info = {
    'gold'  : getDataInfo('gold'),
    'score' : getDataInfo('score')
};

console.info(gridOptions.api.context.data_info);
"""]

custom_css = """
.ag-theme-balham {
    font-size: 10px;
}
.ag-theme-balham .ag-header-cell, .ag-theme-balham .ag-header-group-cell {
    line-height: 24px;
}
.ag-theme-balham .ag-cell {
    line-height: 20px;
    padding-left: 4px;
    padding-right: 4px;
}
.ag-theme-balham .ag-icon-checkbox-unchecked, .ag-theme-balham .ag-icon-checkbox-checked {
    background-size: 12px 12px;
}
.ag-theme-balham .ag-cell-wrapper {
    position: relative;
    top: -2px;
}
.ag-theme-balham .ag-header-cell,
.ag-theme-balham .ag-header-group-cell {
    padding-left: 4px;
    padding-right: 4px;
}
.ag-theme-balham .ag-selection-checkbox span,
.ag-theme-balham .ag-group-expanded span,
.ag-theme-balham .ag-group-contracted span {
    margin-right: 6px;
}
.ag-theme-balham .ag-row-drag {
    width: 20px;
    background-size: 14px 14px;
}
.ag-theme-balham .ag-name-value {
    padding-top: 2px;
    padding-bottom: 2px;
}
.ag-theme-balham .ag-header-cell::after,
.ag-theme-balham .ag-header-group-cell::after {
    margin-top: 4px;
}
.ag-theme-balham .ag-header-cell-menu-button .ag-icon-menu {
    height: 24px;
}
"""

grid_options = {
    'defaultColDef'        : {
        #'editable': True,
        'resizable': True
    },
    'columnDefs'           : columnDefs,
    'headerHeight'         : 24,
    'rowHeight'            : 50,
    'rowDragManaged'       : True,
    'enableSorting'        : True,
    'enableFilter'         : True,
    'enableColResize'      : True,
    'enableRangeSelection' : True,
    'animateRows'          : True,
    'rowGroupPanelShow'    : 'always',
    'sideBar'              : ['columns', 'filters'],
    'statusBar'            : {
        'statusPanels': [
            { 'statusPanel': 'agTotalRowCountComponent', 'align': 'left' },
            { 'statusPanel': 'agFilteredRowCountComponent' },
            { 'statusPanel': 'agSelectedRowCountComponent' },
            { 'statusPanel': 'agAggregationComponent' }
        ]
    }
}

grid2 = Grid(
    grid_data         = data,
    grid_options      = grid_options,
    js_helpers_custom = js_helpers_custom,
    js_pre_grid       = js_pre_grid,
    js_post_grid      = js_post_grid,
    #export_mode       = 'disabled',
    #show_toggle_edit  = True,
    theme             = 'ag-theme-balham',
    css_rules         = custom_css
)

grid2

Grid(columns_fit='size_to_fit', compress_data=True, css_rules_down=['.ag-theme-balham {font-size: 10px;}', '.a…

In [8]:
grid2.get_selected_rows()

In [9]:
grid2.grid_data_out['rows']

KeyError: 'rows'

In [ ]:
out = widgets.Output(layout={'border': '1px solid black'})

In [ ]:
%%css

.myjqstooltip {
    position: absolute;
    left: 0px;
    top: 0px;
    visibility: hidden;
    background-color: rgba(0,0,0,0.5);
    color: white;
    font: 8px calibri;
    text-align: left;
    white-space: nowrap;
    padding: 2px;
    border: 1px solid white;
    border-radius: 3px;
    z-index: 10000;
    box-shadow: 1px 1px 3px #333;
}
.jqsfield {
    color: white;
    font: 10px arial, san serif;
    text-align: left;
}

input[type=range] {
  height: 19px;
  -webkit-appearance: none;
  margin: 0;
  width: 100%;
}
input[type=range]:focus {
  outline: none;
}
input[type=range]::-webkit-slider-runnable-track {
  width: 100%;
  height: 4px;
  cursor: pointer;
  animate: 0.2s;
  box-shadow: 0px 0px 0px #000000;
  background: #2497E3;
  border-radius: 49px;
  border: 0px solid #000000;
}
input[type=range]::-webkit-slider-thumb {
  box-shadow: 0px 0px 0px #000000;
  border: 1px solid #2497E3;
  height: 12px;
  width: 12px;
  border-radius: 50px;
  background: #A1D0FF;
  cursor: pointer;
  -webkit-appearance: none;
  margin-top: -4.5px;
}
input[type=range]:focus::-webkit-slider-runnable-track {
  background: #2497E3;
}
input[type=range]::-moz-range-track {
  width: 100%;
  height: 4px;
  cursor: pointer;
  animate: 0.2s;
  box-shadow: 0px 0px 0px #000000;
  background: #2497E3;
  border-radius: 49px;
  border: 0px solid #000000;
}
input[type=range]::-moz-range-thumb {
  box-shadow: 0px 0px 0px #000000;
  border: 1px solid #2497E3;
  height: 12px;
  width: 12px;
  border-radius: 50px;
  background: #A1D0FF;
  cursor: pointer;
}
input[type=range]::-ms-track {
  width: 100%;
  height: 4px;
  cursor: pointer;
  animate: 0.2s;
  background: transparent;
  border-color: transparent;
  color: transparent;
}
input[type=range]::-ms-fill-lower {
  background: #2497E3;
  border: 0px solid #000000;
  border-radius: 98px;
  box-shadow: 0px 0px 0px #000000;
}
input[type=range]::-ms-fill-upper {
  background: #2497E3;
  border: 0px solid #000000;
  border-radius: 98px;
  box-shadow: 0px 0px 0px #000000;
}
input[type=range]::-ms-thumb {
  margin-top: 1px;
  box-shadow: 0px 0px 0px #000000;
  border: 1px solid #2497E3;
  height: 12px;
  width: 12px;
  border-radius: 50px;
  background: #A1D0FF;
  cursor: pointer;
}
input[type=range]:focus::-ms-fill-lower {
  background: #2497E3;
}
input[type=range]:focus::-ms-fill-upper {
  background: #2497E3;
}

In [ ]:
from ipywidgets import Button, GridBox, Layout, ButtonStyle

header  = Button(description='Header',
                 layout=Layout(width='auto', grid_area='header'),
                 style=ButtonStyle(button_color='lightblue'))
main    = Button(description='Main',
                 layout=Layout(width='auto', grid_area='main'),
                 style=ButtonStyle(button_color='moccasin'))
sidebar = Button(description='Sidebar',
                 layout=Layout(width='auto', grid_area='sidebar'),
                 style=ButtonStyle(button_color='salmon'))
footer  = Button(description='Footer',
                 layout=Layout(width='auto', grid_area='footer'),
                 style=ButtonStyle(button_color='olive'))

left_box = GridBox(children=[header, main, sidebar, footer],
        layout=Layout(
            width='50%',
            grid_template_rows='auto auto auto',
            grid_template_columns='25% 25% 25% 25%',
            grid_template_areas='''
                "header header header header"
                "main main . sidebar "
                "footer footer footer footer"
            ''')
       )

right_box = GridBox(children=[Button(layout=Layout(width='auto', height='auto'),
                         style=ButtonStyle(button_color='darkseagreen')) for i in range(9)
                 ],
        layout=Layout(
            width='50%',
            grid_template_columns='100px 50px 100px',
            grid_template_rows='80px auto 80px',
            grid_gap='5px 10px')
       )

p3 = widgets.HBox([left_box, right_box, right_box])

In [ ]:
tab = widgets.Tab(icons=['check'] * 3)
tab.children = [grid2, out, p3]
tab.set_title(0, 'DataViewer')
tab.set_title(1, 'Log')
tab.set_title(2, 'Layout')

tab

In [ ]:
#out.clear_output()

In [ ]:
@out.capture()
def on_change_data_out_grid2(change):
    print('on_change_data_out')
    #print(change)
    try:
        #change_df = change['new']['grid']
        change_df = change['new']['rows']
        display(change_df.head(5))
    except:
        pass
    
grid2.observe(on_change_data_out_grid2, names='grid_data_out')

In [ ]:
grid2.get_selected_rows()

In [ ]:
widgets.HTML('<i class="fa fa-check"></i>')

## Linking widgets together

### Updating information in the grid via outside widgets

This example explains how to use `user_params` with other widget changes to update the effect of some action on the grid. 
Here we have a JS function highlight that flashes some cells of the grid when clicking one button on certain conditions depending on a parameter. We would like to choose this parameter via a slider.

We could of course build this slider directly inside the JS of the grid using the `to_eval` input parameter. However building such JS components can sometimes be very tedious. We would prefer to ***use an ipywidget*** to build it, like the IntSlider.



In [ ]:
columnDefs = [
    {'headerName': "Country", 'field': "country", 'width': 120, 'rowGroup': 'true'},
    {'headerName': "Year", 'field': "year", 'width': 90, 'pivot': 'true'},
    {'headerName': "Sport", 'field': "sport", 'width': 110, 'rowGroup': 'true'},
    {'headerName': "Athlete", 'field': "athlete"},
    {'headerName': "Gold", 'field': "gold", 'width': 100, 'aggFunc': 'sum'},
];

gridOptions = {
    'pivotMode': 'true',
    'enableColResize': 'true',
    'columnDefs': columnDefs,
    'enableFilter':'true',
    'enableSorting':'true',
    'animateRows':'true',
};

buttons=[{'name':'Highlight', 'action':"""
        var count = view.gridOptions.api.getDisplayedRowCount();
        for (var i = 0; i<count; i++) {
          var rowNode = view.gridOptions.api.getDisplayedRowAtIndex(i);
          if(rowNode.aggData != null && Object.keys(rowNode.aggData).length > 0){
        var keys = Object.keys(rowNode.aggData);
        var gold = [];
        for (var k = 0; k<keys.length; k++){
          var j = 2*k + 1;
          var prop = "pivot_" + j;
          if(rowNode.aggData[prop] == null){
            rowNode.aggData[prop] = 0;
          }
          gold[k] = rowNode.aggData[prop];
        }
        for(var j=0;j<gold.length - 1;j++){
          // test positive if absolute value of difference of sum of gold medals
          // from one olympiad to the next is greater than or equal to sider_value
          if(Math.abs(gold[j] - gold[j+1]) >= view.model.get('user_params').slider_value){
            var column1 = "pivot_" + (2*j+1);
            var column2 = "pivot_" + (2*(j+1)+1);
            view.gridOptions.api.flashCells({rowNodes: [rowNode], columns: [column1, column2] });
          }
        }}}"""}]


pivot = Grid(quick_filter=True,
             theme='ag-theme-balham',
             compress_data=True,
             menu = {'buttons':buttons},
             grid_options=gridOptions,
             grid_data=data,
             columns_fit="auto",
             user_params={'slider_value':50})
pivot

With this function in `buttons`, two adjacent cells are highlighted if their difference in absolute value is higher than `user_parameter.slider_value`.

In [ ]:
# Setting a simple slider to coordinate its value with the highlight button
slider = widgets.IntSlider(
    value=50,
    min=0,
    max=100,
    step=1,
    description='Highlight Value:',
    style={'description_width': 'initial'},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

def on_slider_change(change):
    pivot.user_params = {'slider_value': change.new}

slider.observe(on_slider_change, names='value')

The custom JS function we built uses `view.model.get('user_params')['slider_value']` instead of the regular hardcoded value. As the value of the slider changes, so does the `user_params`, using a simple observe.

In [ ]:
slider

In [ ]:
# pivot

### Using the interactivity of the grid outputs for interactive plotings

Let's try to do some kind of opposite situation. We are going to link a dataset displayed by a [bqplot](https://github.com/bloomberg/bqplot) widget, with the output of a grid.

In [ ]:
from bqplot import pyplot as plt
from bqplot import (
    Axis, ColorAxis, LinearScale, DateScale, DateColorScale, OrdinalScale,
    OrdinalColorScale, ColorScale, Scatter, Lines, Figure, Tooltip
)

In [ ]:
# Copy pasted from bqplot scatter documentation
price_data = pd.DataFrame(np.cumsum(np.random.randn(150, 2).dot([[1.0, -0.8], [-0.8, 1.0]]), axis=0) + 100,
                          columns=['Security 1', 'Security 2'], index=pd.date_range(start='01-01-2007', periods=150))
size = 100
np.random.seed(0)
x_data = range(size)
y_data = np.cumsum(np.random.randn(size) * 100.0)
ord_keys = np.array(['A', 'B', 'C', 'D', 'E', 'F'])
ordinal_data = np.random.randint(5, size=size)
symbols = ['Security 1', 'Security 2']

dates_all = price_data.index.values
dates_all_t = dates_all[1:]
sec1_levels = np.array(price_data[symbols[0]].values.flatten())
log_sec1 = np.log(sec1_levels)
sec1_returns = log_sec1[1:] - log_sec1[:-1]

sec2_levels = np.array(price_data[symbols[1]].values.flatten())

In [ ]:
df = pd.DataFrame({'date':dates_all,'sec_level':sec2_levels})

For the grid, we choose button-export to be able to export the entire grid at will, as soon as the data changes because of filtering.

In [ ]:
column_defs = [
    {'field':'date', 'headerName':'Date', 'rowDrag': True, 'checkboxSelection': True},
    {'field':'sec_level', 'headerName':'Value'}
]

gridOptions={
    'columnDefs'           : column_defs,
    'rowDragManaged'       : True,
    'enableSorting'        : True,
    'enableFilter'         : True,
    'enableColResize'      : True,
    'enableRangeSelection' : True,
    'animateRows'          : True,
    'sideBar'              : 'columns'
};

process_data_grid = Grid(width=600,
             quick_filter=True,
             export_mode="buttons",
             show_toggle_edit=True,
             sync_on_edit=True,
             theme='ag-theme-balham',
             menu={'input_div_css':{'flex-direction':'column', 'flex-wrap':'nowrap'}},
             grid_options=gridOptions,
             grid_data=df)

In [ ]:
sc_x = DateScale()
sc_y = LinearScale()

scatt = Scatter(x=dates_all, y=sec2_levels, scales={'x': sc_x, 'y': sc_y})
ax_x = Axis(scale=sc_x, label='Date')
ax_y = Axis(scale=sc_y, orientation='vertical', tick_format='0.0f', label='Security 2')

Here is the output of the figure :

In [ ]:
Figure(marks=[scatt], axes=[ax_x, ax_y])

In [ ]:
process_data_grid

Now let's observe the `grid_data_out` and update the data in the figure as soon as we get the data out. You can try to filter on any value, especially on the second value to eliminate the records that are lower than a certain value for example.

In [ ]:
def on_change_data_out(change):
    print('on_change_data_out')
    print(change)
    print('XXX')
    try:
        #change_df = change['new']['grid']
        change_df = change['new']['rows']
        display(change_df.head(5))
        x = change_df['date'].tolist()
        y = change_df['sec_level'].tolist()
        x = np.array(x, dtype='datetime64[ns]')
        scatt.x = x
        scatt.y = y
    except:
        pass
    
process_data_grid.observe(on_change_data_out, names='grid_data_out')
#process_data_grid.observe(on_change_data_out)

In [ ]:
process_data_grid.get_selected_rows()

In [ ]:
from string import Template
from IPython.display import HTML, Javascript, display
from IPython.core.magic import register_line_magic, register_cell_magic, register_line_cell_magic

@register_cell_magic
def jsx(line, cell):
    display(Javascript((Template('''
        require(['babel', 'react', 'react-dom'], (Babel, React, ReactDOM) => {
            eval(Babel.transform($quoted_script, {presets: ['react']}).code)
        })
    ''').substitute(quoted_script=json.dumps(cell)))))
    
@register_cell_magic
def css(line, cell):
    display(HTML((Template("<style>$quoted_script</style>").substitute(quoted_script=cell))))

@register_cell_magic
def load_js(line, cell):
    for idx, js_file in enumerate(filter(None, line.split() + cell.split())):
        with open(js_file, encoding='utf-8') as f:
            print('Load[%d]: %s' % (idx, js_file))
            display(Javascript(f.read()))
            
@register_cell_magic
def load_css(line, cell):
    for idx, css_file in enumerate(filter(None, line.split() + cell.split())):
        with open(css_file, encoding='utf-8') as f:
            print('Load[%d]: %s' % (idx, css_file))
            display(HTML((Template("<style>$quoted_script</style>").substitute(quoted_script=f.read()))))
            
@register_line_cell_magic
def load_html(line, cell=""):
    for idx, html_file in enumerate(filter(None, line.split() + cell.split())):
        with open(html_file, encoding='utf-8') as f:
            print('Load[%d]: %s' % (idx, html_file))
            display(HTML(f.read()))

In [ ]:
%%load_js

./js/jquery.sparkline@2.1.2.js